1. A function can be used as tool directly with AssistentAgent by using it like `tools=[func_name]`
2. A tool is mainly defined by importing `FunctionTool` class from `autogen_core.tools` module.

In [ ]:
# import required libraries
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool
import datetime

# Create a function which will be used as a tool
async def get_current_datetime() -> str:
    """Gets the current date and time."""
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)

In [ ]:
# Function can be used as tool like below by directly specifying the function name
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_current_datetime],
    system_message="Use tools to solve tasks.",
)

# Or function can be used with tool with the help of FunctionTool
# This step happens automatically inside the AssistantAgent if the tool is a Python function.
get_datetime_tool = FunctionTool(get_current_datetime, description="Gets the current date and time")

agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_datetime_tool],
    system_message="Use tools to solve tasks.",
)

**`reflect_on_tool_use`** : `bool`, optional

* If `True`, the agent will perform an additional model inference. This inference uses the tool call and its result to generate a final response.
* If `False`, the result of the tool call will be returned directly as the response.
* Defaults to `False`.

In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_datetime_tool],
    system_message="Use tools to solve tasks.",
    reflect_on_tool_use=False
)
response = await agent.on_messages(
    messages=[TextMessage(content="What is the current date and time?", source="user")],
    cancellation_token=CancellationToken(),
)
print("Final Response:", response.chat_message.content)

In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_datetime_tool],
    system_message="Use tools to solve tasks.",
    reflect_on_tool_use=True
)
response = await agent.on_messages(
    messages=[TextMessage(content="What is the current date and time?", source="user")],
    cancellation_token=CancellationToken(),
)
print("Final Response:", response.chat_message.content)

**Important Note about Tool calling and System message:**

The `o1-preview` and `o1-mini` models have the following limitations:

* **System Message:** They do not support system messages. Therefore, the `system_message` parameter must be set to `None`.
* **Function Calling:** They do not support function calling. Consequently, the `tools` and `handoffs` parameters should not be set.